In [1]:
import numpy as np
import pandas as pd

In [2]:
import requests
from bs4 import BeautifulSoup

In [14]:
import re

In [97]:
from tqdm import tqdm

In [3]:
r = requests.get("https://etrain.info/train/14205/history")

In [4]:
soup = BeautifulSoup(r.content, "html.parser")

In [6]:
stations = soup.find_all("a", class_="runStatStn")

In [7]:
len(stations)

11

In [11]:
r.url

'https://etrain.info/train/Fd-Dli-Exp-14205/history'

In [12]:
stations[0]

<a class="runStatStn blocklink rnd5" etitle="&lt;%showRunStatStn(this)%&gt;" href="#AYC">
<div>
								AYODHYA CANT (AYC)
								<div class="inlineblock fltright">
<div class="inlineblock pdl5"><b>Avg. Delay:</b> 1 Min's</div>
</div>
</div>
<div class="nowrap fullw">
<div class="inlineblock" style="background-color:#008000;height:20px;width:96.67%"></div><div class="inlineblock" style="background-color:#ffa500;height:20px;width:3.33%"></div><div class="inlineblock" style="background-color:#ff0000;height:20px;width:0%"></div><div class="inlineblock" style="background-color:#808080;height:20px;width:-1.7763568394003E-15%"></div>
</div>
</a>

In [13]:
stations[0].find("div", class_="inlineblock").text

"\nAvg. Delay: 1 Min's\n"

In [15]:
number = re.compile(r"\d+")

In [19]:
re.search(number, stations[0].find("div", class_="inlineblock").text).group()

'1'

In [20]:
r.url + "?d=1m"

'https://etrain.info/train/Fd-Dli-Exp-14205/history?d=1m'

In [21]:
x = requests.get(r.url + "?d=1m")
soup = BeautifulSoup(x.content, "html.parser")
stations = soup.find_all("a", class_="runStatStn")
len(stations)

11

In [27]:
stations[0].text.strip().split("\r\n\t")

['AYODHYA CANT (AYC)', "\t\t\t\t\t\t\t\nAvg. Delay: 1 Min's"]

In [28]:
re.split(r"\s{2,}", stations[0].text.strip())

['AYODHYA CANT (AYC)', "Avg. Delay: 1 Min's"]

In [61]:
import pandas as pd
import json

In [93]:
with open("train.json") as f:
    data = json.load(f)

In [100]:
df = pd.DataFrame(data["trains"])

In [95]:
df.head()

,name,number
0,VLNK THIVIM EXP,00102
1,PNVL MADGAON SPL,00103
2,MAO PANVEL SPL,00104
3,VELANKANNI SPL,00107
4,VELANKANNI SPL,00109


In [77]:
def getAllData(soup):
    stations = soup.find_all("a", class_="runStatStn")
    station_list = []
    delay_list = []
    for i in range(len(stations)):
        station = re.split(r"\s{2,}", stations[i].text.strip())
        station_list.append(station[0])
        delay_list.append(re.search(r"\d+", station[1]).group())
    return {"station": station_list, "delay": delay_list}

In [90]:
def getTrainData(train_number):
    r = requests.get("https://etrain.info/train/" + train_number + "/history")
    urls = {
        "week": r.url + "?d=1w",
        "month": r.url + "?d=1m",
        "month_3": r.url + "?d=3m",
        "month_6": r.url + "?d=6m",
        "year": r.url + "?d=1y",
    }
    data = {"station": []}
    for day, url in urls.items():
        x = requests.get(url)
        soup = BeautifulSoup(x.content, "html.parser")
        op = getAllData(soup)
        if data["station"] == []:
            data["station"] = op["station"]
        data[day] = op["delay"]

    df = pd.DataFrame(data)
    return df

In [102]:
for train_number in tqdm(df["number"][::-1]):
    new_df = getTrainData(train_number)
    if new_df.shape[0] > 0:
        new_df.to_csv("delay/" + train_number + ".csv", index=False)
    else:
        print(train_number)

  0%|          | 1/5177 [00:01<1:29:34,  1.04s/it]

88888


  0%|          | 2/5177 [00:02<1:33:47,  1.09s/it]

82942


  0%|          | 3/5177 [00:03<1:40:02,  1.16s/it]

...

82734


  0%|          | 20/5177 [00:26<1:44:15,  1.21s/it]

82733


  0%|          | 20/5177 [00:26<1:55:32,  1.34s/it]


KeyboardInterrupt: 